In [2]:
from glob import glob
import os
import subprocess
import nibabel as nib
import numpy as np
import pandas as pd
import utils
import scipy.ndimage
import scipy
import torch
from skimage.measure import regionprops
from monai.networks.blocks import Warp

In [3]:
def getRegionsAsCSV_antss(mask, out):
    ants_command = "ImageMath 3 " + str(out) + " LabelStats " + str(mask) + " " + str(mask)
    subprocess.call(ants_command.split(" "))  
    
def getRegionsAsCSV_scipy(mask, out):    
    img = nib.load(mask).get_fdata().astype(np.uint8)
    regions = regionprops(img)
    df = pd.DataFrame()
    for i, props in enumerate(regions):
        x, y, z = props.centroid
        x = x # + 1 # +1 because ITK-Snap is between 1-128 and Python 0-127
        y = y # + 1
        z = z # + 1
        t = 0
        label = props.label
        mass = props.area
        volume = props.area
        count = len(props.coords)
        row = {
            'x': x,
            'y': y,
            'z': z,
            't': t,
            'label': label,
            'mass': mass,
            'volume': volume,
            'count': count,
        }
        temp_df = pd.DataFrame(data=row, index=[0])
        df = pd.concat([df, temp_df], ignore_index=True)
    df.to_csv(out, index=False)
    
def get_csv_positive(csv, out):
    df = pd.read_csv(csv, sep=',', header=0)
    df_out = pd.DataFrame()
    for i in range(len(df)):
        row = {
            'x': np.abs(df['x'][i]),
            'y': np.abs(df['y'][i]),
            'z': np.abs(df['z'][i]),
            't': df['t'][i],
            'label': df['label'][i],
            'mass': df['mass'][i],
            'volume': df['volume'][i],
            'count': df['count'][i],
        }  
        temp_df = pd.DataFrame(data=row, index=[0])
        df_out = pd.concat([df_out, temp_df], ignore_index=True)
    df_out.to_csv(out, index=False)


In [4]:
ref = os.path.join("/home/valentini/dev/Mousenet/dataset3/Atlas/", "P56_Atlas_128_norm_id.nii.gz")
out_atlas_antss = os.path.join("dataset3", "Atlas", "P56_Atlas_landmarks_id_ants.csv")
out_atlas_scipy = os.path.join("dataset3", "Atlas", "P56_Atlas_landmarks_id_scipy.csv")
atlas_landmark = os.path.join("dataset3", "Atlas", "P56_Atlas_landmarks_id.nii.gz")
#getRegionsAsCSV_antss(atlas_landmark, out_atlas_antss)       
#getRegionsAsCSV_scipy(atlas_landmark, out_atlas_scipy) 

In [5]:
#for dataset in ["Feminad"]:
#    print(dataset)
#    data_dir = os.path.join("/home/valentini/dev/Mousenet/dataset2/" + str(dataset) + "/") 
#    landmarks_origin = sorted(glob(os.path.join(data_dir, "Landmarks_Resample", "*.nii.gz")))
#    landmarks_identi = sorted(glob(os.path.join(data_dir, "Landmarks_Resample_Identity", "*.nii.gz")))
#    landmarks_affine = sorted(glob(os.path.join(data_dir, "Landmarks_Resample_Identity_Affine", "*.nii.gz")))
#    landmarks_deform = sorted(glob(os.path.join(data_dir, "Landmarks_Resample_Identity_Affine_Deformable", "*.nii.gz")))
#    for i in range(len(landmarks_origin)):
#                        
#        out_identi_antss = os.path.join(data_dir, "Landmarks_Resample_Identity", landmarks_identi[i].split('/')[-1].split('.')[0] + '_antss.csv')
#        out_identi_scipy = os.path.join(data_dir, "Landmarks_Resample_Identity", landmarks_identi[i].split('/')[-1].split('.')[0] + '_scipy.csv')
#        getRegionsAsCSV_antss(landmarks_identi[i], out_identi_antss)       
#        getRegionsAsCSV_scipy(landmarks_identi[i], out_identi_scipy)
#                
#        name = landmarks_origin[i].split('/')[-1].split('.')[0]
#        name = "_" + name.split('_')[-2] + '_' + name.split('_')[-1]
#        
#        mat = glob(os.path.join(data_dir, 'MRI_N4_Resample_Norm_Identity_Affine_Deformable', "*" + name + "*.mat"))[0]     
#        out_affine_apply = os.path.join(data_dir, "Landmarks_Resample_Identity_Affine", landmarks_affine[i].split('/')[-1].split('.')[0] + '_apply.csv')
#        ants_command = "antsApplyTransformsToPoints -d 3 -i " + str(out_identi_antss) + " -o " + str(out_affine_apply) + " -t [" + str(mat) + ",1]"
#        subprocess.call(ants_command.split(" "))  
#        
#        out_affine_apply_2 = os.path.join(data_dir, "Landmarks_Resample_Identity_Affine", landmarks_affine[i].split('/')[-1].split('.')[0] + '_apply_pos.csv')
#        get_csv_positive(out_affine_apply, out_affine_apply_2)
#        
#        invwarp = sorted(glob(os.path.join(data_dir, 'MRI_N4_Resample_Norm_Identity_Affine_Deformable', "*" + name + "*InverseWarp.nii.gz")))[0] 
#        warp = sorted(glob(os.path.join(data_dir, 'MRI_N4_Resample_Norm_Identity_Affine_Deformable', "*" + name + "*Warp.nii.gz")))[1]
#        
#        out_deform_apply2 = os.path.join(data_dir, "Landmarks_Resample_Identity_Affine_Deformable", landmarks_deform[i].split('/')[-1].split('.')[0] + '_applytoapply.csv')
#        ants_command = "antsApplyTransformsToPoints -d 3 -i " + str(out_affine_apply) + " -o " + str(out_deform_apply2) + " -t " + str(invwarp)
#        subprocess.call(ants_command.split(" "))
#        
#        out_deform_apply3 = os.path.join(data_dir, "Landmarks_Resample_Identity_Affine_Deformable", landmarks_deform[i].split('/')[-1].split('.')[0] + '_applytoapply_pos.csv')
#        get_csv_positive(out_deform_apply2, out_deform_apply3)
#        
#        out_deform_apply_atlas = os.path.join(data_dir, "Landmarks_Resample_Identity_Affine", landmarks_affine[i].split('/')[-1].split('.')[0] + '_atlas_ants.csv')
#        ants_command = "antsApplyTransformsToPoints -d 3 -i " + str(out_atlas_antss) + " -o " + str(out_deform_apply_atlas) + " -t " + str(warp)
#        subprocess.call(ants_command.split(" "))
#                
#        #out_deform_apply_atlas_2 = os.path.join(data_dir, "Landmarks_Resample_Identity_Affine", landmarks_affine[i].split('/')[-1].split('.')[0] + '_atlas_applytoapply_pos.csv')
#        #get_csv_positive(out_deform_apply_atlas, out_deform_apply_atlas_2)
#        
#    print('-'*40)

## Get DL Landmarks

In [6]:
def interpolate_warp(warp_data, point):
    x_range = np.arange(0, warp_data.shape[1], 1)
    y_range = np.arange(0, warp_data.shape[2], 1)
    z_range = np.arange(0, warp_data.shape[3], 1)

    interp_x = scipy.interpolate.RegularGridInterpolator((x_range, y_range, z_range), warp_data[0, :, :, :],
                                                         fill_value=0)
    dx = interp_x(point)

    interp_y = scipy.interpolate.RegularGridInterpolator((x_range, y_range, z_range), warp_data[1, :, :, :],
                                                         fill_value=0)
    dy = interp_y(point)

    interp_z = scipy.interpolate.RegularGridInterpolator((x_range, y_range, z_range), warp_data[2, :, :, :],
                                                         fill_value=0)
    dz = interp_z(point)

    return dx, dy, dz

def create_row_csv(x, y, z, t, label, mass, volume, count):
    row = {
        'x': x,
        'y': y,
        'z': z,
        't': t,
        'label': label,
        'mass': mass,
        'volume': volume,
        'count': count,
    }
    return row

def apply_warp_to_landmarks_df(df, warp):
    # DDF in [N,N,N,1,3] -> [3,N,N,N]
    if isinstance(warp, str):
        warp = nib.load(warp)
        warp_data = torch.from_numpy(warp.get_fdata())
    elif isinstance(warp, MetaTensor):
        warp_data = warp
    else:
        warp_data = torch.from_numpy(warp.get_fdata())
    warp_data = warp_data.permute(4,0,1,2,3).squeeze().cpu().detach().numpy()
    if isinstance(df, str):
        df = pd.read_csv(df)
    new_df = pd.DataFrame()
    for i in range(len(df)):
        x = df['x'][i]
        y = df['y'][i]
        z = df['z'][i]

        dx, dy, dz = interpolate_warp(warp_data, (x, y, z))
        
        #here -dx and -dy because we saved ddf in ants format (with *[-1,-1,0])
        new_x = x - dx
        new_y = y - dy
        new_z = z + dz

        t = df['t'][i]
        label = df['label'][i]
        mass = df['mass'][i]
        volume = df['volume'][i]
        count = df['count'][i]

        row = create_row_csv(new_x, new_y, new_z, t, label, mass, volume, count)

        temp_df = pd.DataFrame(data=row, index=[0])
        new_df = pd.concat([new_df, temp_df], ignore_index=True)

    return new_df

In [10]:
warp_nearest = Warp("nearest")
for dataset in ["Feminad"]:
    data_dir = os.path.join("/home/valentini/dev/Mousenet/dataset3/" + str(dataset) + "/")
    affine_csvs = sorted(glob(os.path.join(data_dir, "Landmarks", "*_affpos.csv")))
    for output_folder in [
              "feminad-sym-1",
              "feminad-sym-8",
             ]:
        landmarks_folder = os.path.join("output", "Feminad", output_folder, "Landmarks_Deformable")
        deform_warps = sorted(glob(os.path.join("output", "Feminad", output_folder, "DeformableWarp", "*.nii.gz")))
        try:
            os.mkdir(landmarks_folder)
        except Exception:
            pass         
        for i, csv in enumerate(affine_csvs):
            print(str(i+1) + "/" + str(len(affine_csvs)), end='\r')
            name = csv.split('/')[-1].split('.')[0]
            name = name.split('_')[0] + '_' + name.split('_')[1]
            deform_warp = deform_warps[i]
            
            out_csv_atlas = os.path.join(landmarks_folder, "Affine_" + name + '_atlas.csv')
            apply_warp_to_landmarks_df(out_atlas_scipy, deform_warp).to_csv(out_csv_atlas, index=False)
            
            

monai.networks.blocks.Warp: Using PyTorch native grid_sample.


## Compute Distances

In [7]:
def compute_csv_distance(csv1, csv2):
    if isinstance(csv1, str):
        csv1 = pd.read_csv(csv1, sep=',', header=0)
    if isinstance(csv2, str):
        csv2 = pd.read_csv(csv2, sep=',', header=0)
    TREs = []
    for i in csv2['label']:
        if True:
            ind1 = np.where(csv1['label'] == i)[0]
            ind2 = np.where(csv2['label'] == i)[0]
            if len(ind1) == 1 and len(ind2) == 1:
                x1 = csv1['x'][ind1[0]]
                x2 = csv2['x'][ind2[0]]

                y1 = csv1['y'][ind1[0]]
                y2 = csv2['y'][ind2[0]]

                z1 = csv1['z'][ind1[0]]
                z2 = csv2['z'][ind2[0]]

                TRE = np.sqrt((x1-x2)*(x1-x2)+(y1-y2)*(y1-y2)+(z1-z2)*(z1-z2))
                TREs.append(TRE)
    return TREs

In [10]:
for dataset in ["Feminad"]:

    for output_folder in [
              "lambda1_0.001",
              "lambda1_0.01",
              "lambda1_0.1",
              "lambda1_1.0",
              "lambda1_10",
              "lambda1_100",
              "lambda1_1000",
              "scenario1-final",
              "scenario2-final",
              "scenario3-final",
              "feminad-sym-1",
              "feminad-sym-8",
    ]:
        
        original_vals = [[], [], [], [], [], []]
        affine_vals = [[], [], [], [], [], []]
        ants_vals = [[], [], [], [], [], []]
        output_vals = [[], [], [], [], [], []]
        
        landmarks_deep_atlas = sorted(glob(os.path.join("output", "Feminad", output_folder, "Landmarks_Deformable", "*_atlas.csv")))
                       
        landmarks_original_apply = sorted(glob(os.path.join("dataset3", "Feminad", "Landmarks", "*_og.csv")))
        landmarks_original_apply_pos = sorted(glob(os.path.join("dataset3", "Feminad", "Landmarks", "*_ogpos.csv")))

        landmarks_affine_apply = sorted(glob(os.path.join("dataset3", "Feminad", "Landmarks", "*_aff.csv")))
        landmarks_affine_apply_pos = sorted(glob(os.path.join("dataset3", "Feminad", "Landmarks", "*_affpos.csv")))

        landmarks_ants_atlas = sorted(glob(os.path.join("dataset3", "Feminad", "Landmarks", "*_atlasdef.csv"))) 
        landmarks_ants_atlaspos = sorted(glob(os.path.join("dataset3", "Feminad", "Landmarks", "*_atlasdefpos.csv"))) 
        
        #print(landmarks_deep_atlas)

        landmarks_template_antss = os.path.join("dataset3", "Atlas", "P56_Atlas_landmarks_id_ants.csv")
        landmarks_template_scipy = os.path.join("dataset3", "Atlas", "P56_Atlas_landmarks_id_scipy.csv")
        
        for i, landmark in enumerate(landmarks_affine_apply):
            
            original_apply = compute_csv_distance(landmarks_original_apply[i], landmarks_template_antss)            
            affine_apply = compute_csv_distance(landmarks_affine_apply[i], landmarks_template_antss)
            ants_atlas = compute_csv_distance(landmarks_affine_apply[i], landmarks_ants_atlas[i])            
            deep_atlas = compute_csv_distance(landmarks_affine_apply_pos[i], landmarks_deep_atlas[i])
            
            for i in range(len(original_apply)):
                original_vals[i].append(original_apply[i])
                affine_vals[i].append(affine_apply[i])
                ants_vals[i].append(ants_atlas[i])
                output_vals[i].append(deep_atlas[i])        
                
        means = []
        stds = []
        #output_vals = np.array(output_vals)
        #output_vals = output_vals[1:6,:]
        #print(output_folder)
        #print(np.mean(output_vals,0))
        for val in output_vals:
            means.append(np.mean(val))
            stds.append(np.std(val))
        means = means[1:6]
        stds = stds[1:6]
        #print(means)
        #print(stds)
        #print(output_folder + ' - ' + str(means) + ' - ' + str(stds))   
        print(output_folder + ' - ' + str(np.mean(means)) + ' - ' + str(np.mean(stds)))      
    names = ["ants", "affine", "original"]
    for i, liste in enumerate([ants_vals, affine_vals, original_vals]):            
        means = []
        stds = []
        for val in liste:
            means.append(np.mean(val))
            stds.append(np.std(val))
        means = means[1:6]
        stds = stds[1:6]
        #print(names[i])
        #print(means)
        #print(stds)
        #print(names[i] + ' - ' + str(means) + ' - ' + str(stds))  
        print(names[i] + ' - ' + str(np.mean(means)) + ' - ' + str(np.mean(stds)))  
            

lambda1_0.001 - 31.966334436703317 - 0.931578258486779
lambda1_0.01 - 29.141472785019765 - 1.0455574776124805
lambda1_0.1 - 3.5189470711006217 - 0.8135360261722824
lambda1_1.0 - 2.869457274790764 - 0.7000282160789857
lambda1_10 - 2.8854478758141577 - 0.7302917058655005
lambda1_100 - 3.577687011055823 - 0.8744040220581096
lambda1_1000 - 3.7731739400647704 - 0.9269059051238354
scenario1-final - 2.778124207827283 - 0.7226061397678561
scenario2-final - 2.903131910227366 - 0.7142292005744381
scenario3-final - 2.878842531230274 - 0.7094434315746156
feminad-sym-1 - 3.820854776002534 - 1.3973309790218464
feminad-sym-8 - 3.661152420936677 - 1.3504505306651176
ants - 2.9413065951480224 - 1.0414585258034161
affine - 3.5469232304627183 - 0.8772366850714087
original - 21.072112832313785 - 2.813962657620705


In [14]:
for dataset in ["Feminad"]:

    for output_folder in [
              "lambda1_0.001",
              "lambda1_0.01",
              "lambda1_0.1",
              "lambda1_1.0",
              "lambda1_10",
              "lambda1_100",
              "lambda1_1000",
              "scenario1-final",
              #"scenario2-final",
              #"scenario3-final",
              #"feminad-sym-1",
              "feminad-sym-8",
    ]:
        
        original_vals = [[], [], [], [], [], []]
        affine_vals = [[], [], [], [], [], []]
        ants_vals = [[], [], [], [], [], []]
        output_vals = [[], [], [], [], [], []]
        landmarks_deep_atlas = sorted(glob(os.path.join("output", "Feminad", output_folder, "Landmarks_Deformable", "*_atlasfv.csv")))
                       
        landmarks_original_apply = sorted(glob(os.path.join("dataset3", "Feminad", "Landmarks", "*_og.csv")))
        landmarks_original_apply_pos = sorted(glob(os.path.join("dataset3", "Feminad", "Landmarks", "*_ogpos.csv")))

        landmarks_affine_apply = sorted(glob(os.path.join("dataset3", "Feminad", "Landmarks", "*_aff.csv")))
        landmarks_affine_apply_pos = sorted(glob(os.path.join("dataset3", "Feminad", "Landmarks", "*_affpos.csv")))

        landmarks_ants_atlas = sorted(glob(os.path.join("dataset3", "Feminad_Mean", "Landmarks", "*_atlasfvdef.csv"))) 
        landmarks_ants_atlaspos = sorted(glob(os.path.join("dataset3", "Feminad_Mean", "Landmarks", "*_atlasfvdefpos.csv"))) 

        landmarks_template_antss = os.path.join("dataset3", "Atlas", "P56_Atlas_landmarks_FV_id_ants.csv")
        
        for i, landmark in enumerate(landmarks_affine_apply):
            
            original_apply = compute_csv_distance(landmarks_original_apply[i], landmarks_template_antss)            
            affine_apply = compute_csv_distance(landmarks_affine_apply[i], landmarks_template_antss)
            ants_atlas = compute_csv_distance(landmarks_affine_apply[i], landmarks_ants_atlas[i])            
            deep_atlas = compute_csv_distance(landmarks_affine_apply_pos[i], landmarks_deep_atlas[i])
            
            for i in range(len(original_apply)):
                original_vals[i].append(original_apply[i])
                affine_vals[i].append(affine_apply[i])
                ants_vals[i].append(ants_atlas[i])
                output_vals[i].append(deep_atlas[i])        
                
        means = []
        stds = []
        for val in output_vals:
            means.append(np.mean(val))
            stds.append(np.std(val))
        means = means[1:6]
        stds = stds[1:6]
        print(output_folder)
        print(means)
        #print(output_folder + ' - ' + str(means) + ' - ' + str(stds))     
        #print(output_folder + ' - ' + str(np.mean(means)) + ' - ' + str(np.mean(stds)))   
    names = ["ants", "affine", "original"]
    #for i, liste in enumerate([ants_vals, affine_vals, original_vals]):            
    #    means = []
    #    stds = []
    #    for val in liste:
    #        means.append(np.mean(val))
    #        stds.append(np.std(val))        
    #    means = means[1:6]
    #    stds = stds[1:6]
    #    #print(names[i] + ' - ' + str(means) + ' - ' + str(stds))  
    #    print(names[i] + ' - ' + str(np.mean(means)) + ' - ' + str(np.mean(stds)))
            

lambda1_0.001
[51.01070705674939, 37.864855609996155, 29.26920236393829, 13.57711875199628, 28.074136111305865]
lambda1_0.01
[49.053984318193734, 35.431126917481016, 26.499286074988518, 23.87249609300217, 10.555083485756427]
lambda1_0.1
[5.455809223980501, 1.5928976444347016, 1.8808002829830166, 1.9078989487497486, 2.0057307630705115]
lambda1_1.0
[3.5331696782702493, 1.6507441086573098, 1.6602327622634068, 1.7656589902875082, 2.25811723655985]
lambda1_10
[3.1020918752971784, 1.5199767063889764, 1.631083720057024, 2.2057133866324112, 2.439064237861165]
lambda1_100
[3.385433937406982, 2.170256734571169, 2.4432917930096325, 3.753442693383559, 4.020586321562468]
lambda1_1000
[2.906158515377511, 2.092670885128756, 2.7213352914128373, 4.359094576702778, 4.616174391337452]
scenario1-final
[3.0694454190926033, 1.5123403290305377, 1.5988052978782135, 1.970313420550443, 2.2213181572691614]
feminad-sym-8
[3.0137911709591982, 1.9777172152447167, 2.439506915333444, 4.05570321204753, 4.1286485217043

In [ ]:
lambda1_0.001 - 31.959203978797195 - 0.9328134737368637
lambda1_0.01 - 29.08239537788437 - 1.0314044549371026
lambda1_0.1 - 2.5686273726436957 - 0.7726743581931246
lambda1_1.0 - 2.1735845552076647 - 0.7118793627565979
lambda1_10 - 2.179585985247351 - 0.7591155726078209
lambda1_100 - 3.154602295986762 - 0.8578874299380533
lambda1_1000 - 3.339086731991867 - 0.8887409659368842
scenario1-final - 2.074444524764192 - 0.7537320502894346
scenario2-final - 2.1808060413345096 - 0.7478311830632397
scenario3-final - 2.1696611992521873 - 0.7424738803593313
ants - 2.560370631247044 - 0.7888005771213398
affine - 3.0201340831860612 - 0.862368130215337
original - 20.469665982422107 - 2.8179641100049286

In [ ]:
scenario1-final - [6.5878438344080585, 1.5123403290305377, 1.5988052978782135, 1.970313420550443, 2.2213181572691614] - [0.8914072774541518, 0.7137436777322729, 0.6253148536275893, 0.8312285825117052, 0.5513363075135613]
scenario2-final - [6.774895849309567, 1.465030623355497, 1.6152106717630432, 2.1351770268371033, 2.5253453798716197] - [0.8811999719012064, 0.7043029984513115, 0.6380589962740504, 0.8093591533607786, 0.5382248828848436]
scenario3-final - [6.650890157591889, 1.549108442915863, 1.611179004555159, 2.152562144202721, 2.430472906885737] - [0.8852039913801888, 0.700335999125299, 0.6635909988962887, 0.7913784681360218, 0.5067077003352796]
ants - [3.370438071499331, 1.769509809151483, 2.322182013352659, 4.2048299741948245, 3.039573107541815] - [0.9641717997956609, 0.7593787168598513, 1.2437284675940392, 1.41945954522733, 0.8205540995402001]
affine - [5.608334863531274, 1.841422897568715, 2.3066097505865324, 3.9236349040985066, 4.054613736528563] - [1.0502172756508101, 0.7839124805474929, 0.912029660111654, 0.9511467945561554, 0.6888772144909318]
original - [17.03725706225109, 17.307953884339863, 19.525314660564106, 25.582794914058454, 25.90724364035541] - [2.796923467970565, 2.7488078249991856, 2.5354987818232417, 2.924193464107327, 3.064389749203205]

In [ ]:
lambda1_0.001 - [51.046359346279964, 37.864855609996155, 29.26920236393829, 13.57711875199628, 28.074136111305865] - [1.1010818760075238, 1.0813650529789733, 1.1195845069393024, 0.6817024360731896, 0.6741574204349057]
lambda1_0.01 - [49.34937135387071, 35.431126917481016, 26.499286074988518, 23.87249609300217, 10.555083485756427] - [1.0573208436683577, 1.1273376933019814, 1.3881360073061189, 0.9488527757238596, 0.7061400680620852]
lambda1_0.1 - [10.207407716265129, 1.5928976444347016, 1.8808002829830166, 1.9078989487497486, 2.0057307630705115] - [1.1801123449308357, 0.6964035406010131, 0.8831246933567831, 0.8899821963376857, 0.4180573556350943]
lambda1_1.0 - [7.012533276185746, 1.6507441086573098, 1.6602327622634068, 1.7656589902875082, 2.25811723655985] - [0.9523287546433415, 0.6920191586174759, 0.6669454065705062, 0.73524151366816, 0.45360624689544454]
lambda1_10 - [6.631401328131212, 1.5199767063889764, 1.631083720057024, 2.2057133866324112, 2.439064237861165] - [0.8820022020511263, 0.7225225074446215, 0.6334717767781383, 0.8592440316030459, 0.5542180114505704]
lambda1_100 - [5.500857512752287, 2.170256734571169, 2.4432917930096325, 3.753442693383559, 4.020586321562468] - [1.0736428271007479, 0.8209741575796928, 0.8176741633572873, 0.9420852470012151, 0.7176437152516052]
lambda1_1000 - [5.076594555742029, 2.092670885128756, 2.7213352914128373, 4.359094576702778, 4.616174391337452] - [1.0668027151049542, 0.8246096363203339, 1.0770944104121116, 0.9472037081148217, 0.7188190556669557]

In [ ]:
lambda1_0.001 - 31.966334436703317 - 0.931578258486779
lambda1_0.01 - 29.141472785019765 - 1.0455574776124805
lambda1_0.1 - 3.5189470711006217 - 0.8135360261722824
lambda1_1.0 - 2.869457274790764 - 0.7000282160789857
lambda1_10 - 2.8854478758141577 - 0.7302917058655005
lambda1_100 - 3.577687011055823 - 0.8744040220581096
lambda1_1000 - 3.7731739400647704 - 0.9269059051238354

In [ ]:
scenario1-0.1: 3.3148086915925483
scenario2-0.1: 4.786604332016063
scenario3-0.1: 4.92144044156853
scenario1-8.0: 2.785438264714081
scenario2-8.0: 2.8608613936924794
scenario3-8.0: 2.7818941269866193
ants2: 2.9413065951480237
affine: 3.5469232304627174
original: 21.072112832313785

# Vérification

In [79]:
custom_test = np.zeros((128,128,128,1,3))
for i in range(128):
    custom_test[:,:,i,0,2] = i-64
custom_ddf = nib.Nifti1Image(custom_test, nib.load(ref).affine, nib.load(ref).header)  
nib.save(custom_ddf, "dataset2/Feminad/Custom_DDF.nii.gz") 
print(custom_ddf.shape)
print(custom_ddf.get_fdata()[50,50,50,0,:])

(128, 128, 128, 1, 3)
[  0.   0. -14.]


In [82]:
data_dir = os.path.join("/home/valentini/dev/Mousenet/dataset2/Feminad/")
affine_mris = sorted(glob(os.path.join(data_dir, 'MRI_N4_Resample_Norm_Identity_Affine', "*12_6517*.nii.gz")))
#affine_csvs = sorted(glob(os.path.join(data_dir, 'Landmarks_Resample_Identity_Affine', "*12_6517*apply.csv")))
affine_csvs_pos = sorted(glob(os.path.join(data_dir, 'Landmarks_Resample_Identity_Affine', "*12_6517*apply_pos.csv")))
#affine_csvs_pos = sorted(glob(os.path.join(data_dir, 'Landmarks_Resample_Identity_Affine', "*12_6517*Custom.csv")))
print(affine_csvs_pos)

outfolder_custom = os.path.join(data_dir, 'Test')
try:
    os.mkdir(outfolder_custom)
except Exception:
     pass        
    
warp = Warp("bilinear", "border")
point = 50
landmarks_template_antss = os.path.join("dataset2", "Atlas", "Identity_Feminad_Template_Landmarks_antss.csv")
landmarks_template_scipy = os.path.join("dataset2", "Atlas", "Identity_Feminad_Template_Landmarks_scipy.csv")
        
for i in range(len(affine_mris)):
    monaiwarp_mri_name = outfolder_custom + "/MonaiWarp_" + affine_mris[i].split('/')[-1]
    mri = torch.from_numpy(nib.load(affine_mris[i]).get_fdata()).unsqueeze(0).unsqueeze(0)
    ddf_good = torch.from_numpy(custom_ddf.get_fdata()).permute(3,4,0,1,2)
    monaiwarp_mri_data = warp(mri, ddf_good).squeeze()
    monaiwarp_mri = nib.Nifti1Image(monaiwarp_mri_data, nib.load(ref).affine, nib.load(ref).header)  
    nib.save(monaiwarp_mri, monaiwarp_mri_name)
    

    print('DDF: ' + str(ddf_good[0,:,point,point,point]))
        
    df_original = pd.read_csv(landmarks_template_scipy)
    print('Original:')
    print(df_original)
    applypos_outname = outfolder_custom + "/applypos_" + affine_mris[i].split('/')[-1].split('.')[0] + '2.csv'
    ants_command = "antsApplyTransformsToPoints -d 3 -i " + str(landmarks_template_antss) + " -o " + str(applypos_outname) + " -t dataset2/Feminad/Custom_DDF.nii.gz"
    subprocess.call(ants_command.split(" "))
    df_outants = pd.read_csv(applypos_outname)    
    print('ANTS ApplyTransformsToPoints:')
    print(df_outants)
    
    test = apply_warp_to_landmarks_df(landmarks_template_scipy, "dataset2/Feminad/Custom_DDF.nii.gz")
    #test.to_csv(outfolder_custom + "/test.csv", index=False)    
    print('Selfmade ApplyTransformsToCSV:')
    print(test)    
 
print('-'*40)
    





['/home/valentini/dev/Mousenet/dataset2/Feminad/Landmarks_Resample_Identity_Affine/Affine_Identity_Landmarks_12_6517_apply_pos.csv']


monai.networks.blocks.Warp: Using PyTorch native grid_sample.


DDF: tensor([  0.,   0., -14.], dtype=torch.float64)
Original:
      x     y      z  t  label  mass  volume  count
0  51.0   4.0   71.0  0      1     1       1      1
1  53.0  43.0  108.0  0      2     1       1      1
2  52.0  53.0   99.0  0      3     1       1      1
3  52.0  65.0   96.0  0      4     1       1      1
4  59.0  84.0   77.0  0      5     1       1      1
5  43.0  84.0   77.0  0      6     1       1      1
6  52.0  65.0   81.0  0      6     1       1      1
ANTS ApplyTransformsToPoints:
    x   y    z  t  label  mass  volume  count
0 -51  -4   71  0      1     1       1      1
1 -53 -43  108  0      2     2       1      1
2 -52 -53   99  0      3     3       1      1
3 -52 -65   96  0      4     4       1      1
4 -59 -84   77  0      5     5       1      1
5 -43 -84   77  0      6     6       1      1
Selfmade ApplyTransformsToCSV:
      x     y      z  t  label  mass  volume  count
0  51.0   4.0   78.0  0      1     1       1      1
1  53.0  43.0  152.0  0      2    

In [415]:
x_image = x_